# Webscraper
Wanneer je extra data nodig hebt om je dataset te vergroten kan je overwegen een webscraper te gebruiken deze extra afbeeldingen te vinden. De Python code waaruit deze notebook is opgebouwd is te vinden in de folder 'webscraper' met daarin oudere versies, de benodigdheden voor het runnen van een webscraper, opnames van het proces, de resultaten van het scrapen en een excel-sheet voor de overwegingen voor het vervolg.

**Note:** Voor de huidige versie wordt gebruik gemaakt van Firefox met de geckodriver (aangezien er geen driver beschikbaar was voor huidige chrome versie). Het is mogelijk chrome te gebruiken maar hiervoor moet je zelf nog `IMAGE_CLASS_NAME` achterhalen voor de afbeeldingen.

#### Global Variables

In [ ]:
PATH = "E:\\Studie\\Stage\\webscraper\\geckodriver.exe" # Path naar driver
FIREFOX_BINARY_PATH = "C:\\Program Files\\Mozilla Firefox\\firefox.exe" # Path naar browser exe
IMAGE_CLASS_NAME = "sFlh5c.FyHeAf.iPVvYb" # Classname for images

#### Imports

In [ ]:
from selenium import webdriver
from selenium.common import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image, ImageTk
import tkinter as tk
import time
import os

### Web-Scraper

In [ ]:
class WebScraper:
    def __init__(self):
        self.wd = None
        self.urls = None
        self.offset_x = None
        self.offset_y = None

    def setup_scraper(self):
        # Instellingen voor Firefox
        options = Options()
        options.binary_location = FIREFOX_BINARY_PATH
        options.headless = False

        # Maak een WebDriver aan zonder gebruik van webdriver_manager
        service = Service(PATH)
        self.wd = webdriver.Firefox(service=service, options=options)
        self.urls = set()

    def open_web(self, URL):
        self.wd.get(URL)
        # Wacht tot het element met het specifieke id beschikbaar is en klik erop
        close_button = WebDriverWait(self.wd, 10).until(
            EC.element_to_be_clickable((By.ID, "W0wltc"))
        )
        close_button.click()

        # Klik op de tumbnail button als het element beschikbaar is
        tumbnail_button = WebDriverWait(self.wd, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".H8Rx8c"))
        )
        tumbnail_button.click()

    def find_image_src(self, class_name, delay=0.2, timeout=10):
        time.sleep(delay)
        try:
            # Gebruik WebDriverWait om te wachten totdat het element beschikbaar is
            WebDriverWait(self.wd, timeout).until(
                EC.presence_of_element_located((By.CLASS_NAME, class_name))
            )

            # Vind Afbeelding
            image_elements = self.wd.find_elements(By.CLASS_NAME, class_name)
            for image in image_elements:
                src = image.get_attribute("src")
                if src and 'http' in src:
                    self.urls.add(src)
                    print("found image:", src)

        except TimeoutException:
            print("Timeout: kon het element met class_name niet vinden binnen de opgegeven tijd.")

        except Exception as e:
            print("Er is een fout opgetreden tijdens het vinden van de afbeelding:\n", e)


    def go_next_image(self):
        if not self.offset_x:
            try:  # find button element
                # Wacht tot de knop met het juiste aria-label klikbaar is en klik erop
                next_button = WebDriverWait(self.wd, 10).until(
                    # EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Volgende afbeelding']"))
                    EC.element_to_be_clickable((By.XPATH, "//button[@jsname='OCpkoe']"))
                )
                # Verkrijg locaties en maten
                location = next_button.location
                size = next_button.size
                print("Button locatie:", location)
                print("Button grootte:", size)

                # Bereken het middelpunt van het element
                self.offset_x = location['x'] + size['width'] / 2
                self.offset_y = location['y'] + size['height'] / 2
                print("x offset:", self.offset_x, "| y offset:", self.offset_y)

            except Exception as e:
                print("Error while finding button:", e)
                # print(self.offset_x, self.offset_y)
        try:
            # Simuleer klik op 'Volgende afbeelding'
            actions = ActionChains(self.wd)
            actions.move_by_offset(self.offset_x,
                                   self.offset_y).click().perform()
            # Move terug
            actions.move_by_offset(self.offset_x * -1,
                                   self.offset_y * -1).perform()
        except Exception as e:
            print("Error while CLICKING next button:", e)

    def get_images(self, max_len=1000):
        counter = 0
        c = 0
        while len(self.urls) < max_len:
            # Scroll to the top
            self.wd.execute_script("window.scrollTo(0, 0)")
            time.sleep(0.15)

            last_len = len(self.urls)

            self.find_image_src(IMAGE_CLASS_NAME, delay=0.1)
            try:
                # Soms heeft het programma niet door dat er een nieuwe src is toegevoegd.
                if last_len == len(self.urls):
                    counter += 1
                else:
                    counter = 0
                if counter == 20:
                    break
            except Exception as e:
                print(e)
            self.go_next_image()
            c += 1
        print("self.urls length:", len(self.urls), "| counter size:", c)
        return self.urls

    def scrape_images(self, class_dict, amount):
        for class_name, URL in class_dict.items():
            self.setup_scraper()
            self.open_web(URL)
            time.sleep(3)

            self.get_images(amount)

            self.wd.execute_script("window.scrollTo(0, document.body.scrollHeight)")

            # Print gevonden URL's
            print("Gevonden URL's:", self.urls)

            self.write_to_file(class_name)
            time.sleep(1)
            self.wd.quit()

    def write_to_file(self, class_name):
        # File path where the set will be saved
        file_path = "img_src_paths\\" + class_name + '.txt'

        # Open the file in write mode
        with open(file_path, 'w') as file:
            for link in self.urls:
                file.write(link + '\n')
        print(f"Set has been saved to {file_path}")
        return True

#### User Input
Maak een `class_dict` met de naam voor het bestand van het zoek resultaat met de bijhorende URL op google images. 

In [ ]:
# Voorbeeld class_dict
class_dict = {
    "naam bestand" : "URL na uivoeren google image search"
}

In [ ]:
# 1 vd class_dicts gebruikt tijdens project
DONT_USE = {
    "sinaasappel" : "https://www.google.com/search?sca_esv=4ebb9fe4becce684&q=sinaasappel&udm=2&fbs=AEQNm0A6bwEop21ehxKWq5cj-cHaxUZOSO72WoU7KkLyB7O1BOnPTqc2lmP5Jtiku-C_ETuS3sociDRHPGNCwJbzshSCdJ_BSgyAonffcmsoASpuek7ub0FxVwE7Kvt4wKYZeIpvWCW5rAj73VwqRL4osU_6y-G7MrjwXqWJ8pSRoOfeoy_deL-scUsp-Md6wGCilrYQJX4b&sa=X&ved=2ahUKEwihmfHEv5qKAxVhgv0HHccxJOcQtKgLegQIFxAB&biw=1229&bih=895",
    "rode_appel" : "https://www.google.com/search?q=rode+appel&sca_esv=4ebb9fe4becce684&udm=2&biw=1229&bih=895&ei=wMhWZ5a2Je-D9u8P-J_foAs&ved=0ahUKEwiWlvTFv5qKAxXvgf0HHfjPF7QQ4dUDCBA&uact=5&oq=rode+appel&gs_lp=EgNpbWciCnJvZGUgYXBwZWwyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAESMMUUL4GWOcQcAF4AJABAJgBT6ABywSqAQIxMLgBA8gBAPgBAZgCC6AChAXCAg0QABiABBixAxhDGIoFwgILEAAYgAQYsQMYgwHCAgYQABgHGB7CAgoQABiABBhDGIoFwgIOEAAYgAQYsQMYgwEYigXCAggQABiABBixA5gDAIgGAZIHAjExoAehMA&sclient=img",
    "groene_appel" : "https://www.google.com/search?q=groene+appel&sca_esv=4ebb9fe4becce684&udm=2&biw=1229&bih=895&ei=_chWZ873Jca49u8PqM72kQo&oq=groene+appel&gs_lp=EgNpbWciDGdyb2VuZSBhcHBlbCoCCAAyChAAGIAEGEMYigUyBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgoQABiABBhDGIoFMgUQABiABEiCNFCTDliRInADeACQAQCYAawBoAGVBqoBAzMuNLgBAcgBAPgBAZgCCaACiQWYAwCIBgGSBwM2LjOgB5Ij&sclient=img",
    "banaan" : "https://www.google.com/search?q=banaan&sca_esv=4ebb9fe4becce684&udm=2&biw=1229&bih=895&ei=H8lWZ5y7BqL97_UP6reP0AY&ved=0ahUKEwjcxvvyv5qKAxWi_rsIHerbA2oQ4dUDCBA&uact=5&oq=banaan&gs_lp=EgNpbWciBmJhbmFhbjINEAAYgAQYsQMYQxiKBTIKEAAYgAQYQxiKBTIFEAAYgAQyDhAAGIAEGLEDGIMBGIoFMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAESJ4WUKcGWKgMcAF4AJABAJgBQaAB1wKqAQE2uAEDyAEA-AEBmAIHoALyAsICBhAAGAcYHsICCBAAGIAEGLEDwgILEAAYgAQYsQMYgwGYAwCIBgGSBwE3oAfeHQ&sclient=img",
    "peer" : "https://www.google.com/search?q=peer&sca_esv=4ebb9fe4becce684&udm=2&biw=1229&bih=895&ei=bslWZ_CCCbX-7_UP-frn0AM&ved=0ahUKEwjw8dOYwJqKAxU1_7sIHXn9GToQ4dUDCBA&uact=5&oq=peer&gs_lp=EgNpbWciBHBlZXIyDRAAGIAEGLEDGEMYigUyCxAAGIAEGLEDGIMBMgsQABiABBixAxiDATIIEAAYgAQYsQMyCBAAGIAEGLEDMgUQABiABDIFEAAYgAQyBRAAGIAEMggQABiABBixAzIFEAAYgARIqw9QnAtYhA5wAXgAkAEAmAGsAaABpwKqAQMyLjG4AQPIAQD4AQGYAgSgArcCwgIGEAAYBxgewgIKEAAYgAQYQxiKBcICDhAAGIAEGLEDGIMBGIoFmAMAiAYBkgcDMy4xoAfYDg&sclient=img",
    "dried_plum" : "https://www.google.com/search?q=dried+plum&sca_esv=4ebb9fe4becce684&udm=2&biw=1229&bih=895&ei=_8lWZ4_tM7KU9u8P8ZytiQs&ved=0ahUKEwjP6JDewJqKAxUyiv0HHXFOK7EQ4dUDCBA&uact=5&oq=dried+plum&gs_lp=EgNpbWciCmRyaWVkIHBsdW0yBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAESI0_ULIKWPc3cAR4AJABAJgBtAGgAaIGqgEEMTIuMbgBA8gBAPgBAZgCEKACzQbCAg0QABiABBixAxhDGIoFwgIIEAAYgAQYsQPCAgYQABgHGB7CAgoQABiABBhDGIoFwgIOEAAYgAQYsQMYgwEYigXCAgsQABiABBixAxiDAcICBxAAGIAEGAqYAwCIBgGSBwQxNS4xoAe9OA&sclient=img",
    "cherry_pruimtomaat" : "https://www.google.com/search?client=firefox-b-d&sca_esv=4ebb9fe4becce684&udm=2&q=cherry+pruimtomaat&spell=1&sa=X&ved=2ahUKEwimqtSxwZqKAxUC8LsIHf1YOJ8QBSgAegQIBxAB&biw=1229&bih=895&dpr=1",
    "mandarijn" : "https://www.google.com/search?q=mandarijn&client=firefox-b-d&sca_esv=4ebb9fe4becce684&udm=2&biw=1229&bih=895&ei=scpWZ-aSGsf_7_UPnJbdqQo&ved=0ahUKEwimr-eywZqKAxXH_7sIHRxLN6UQ4dUDCBA&uact=5&oq=mandarijn&gs_lp=EgNpbWciCW1hbmRhcmlqbjIIEAAYgAQYsQMyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgARI8oYCUKX3AVidgAJwA3gAkAEAmAFCoAHfA6oBATm4AQPIAQD4AQGYAgygAooEwgIEEAAYHsICCxAAGIAEGLEDGIMBwgIOEAAYgAQYsQMYgwEYigXCAgoQABiABBhDGIoFwgIQEAAYgAQYsQMYQxiDARiKBZgDAIgGAZIHAjEyoAfUKw&sclient=img",
    "green_kiwi" : "https://www.google.com/search?q=green+kiwi&client=firefox-b-d&sca_esv=4ebb9fe4becce684&udm=2&biw=1229&bih=895&ei=18pWZ6e-OomA9u8Px_fG2QE&ved=0ahUKEwjnhZfFwZqKAxUJgP0HHce7MRsQ4dUDCBA&uact=5&oq=green+kiwi&gs_lp=EgNpbWciCmdyZWVuIGtpd2kyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAESIstUL8EWK8ZcAR4AJABAJgBPqABlgWqAQIxM7gBA8gBAPgBAZgCEaAC6AXCAg0QABiABBixAxhDGIoFwgIGEAAYBxgewgIKEAAYgAQYQxiKBcICCxAAGIAEGLEDGIMBwgIOEAAYgAQYsQMYgwEYigXCAggQABiABBixA8ICBBAAGAPCAgoQABiABBixAxgKwgIHEAAYgAQYCpgDAIgGAZIHAjE3oAf1QA&sclient=img"
}

Verander `num_imgs` naar de hoeveelheid afbeeldingen die je wilt. Dit is zonder het meerekenen van afbeeldingen met beschermde auteursrechten. Vaak wordt het maximum niet behaald aangezien er geen verdere afbeeldingen te vinden zijn.

In [ ]:
# Maximaal aantal afbeeldingen
num_imgs = 1000

In [ ]:
ws = WebScraper()
ws.scrape_images(class_dict, num_imgs)

### Download Images
Na het ophalen van alle source links naar de afbeeldingen bezoeken we deze links en slaan (als mogelijk) deze afbeeldingen op.

In [ ]:
def download_images_from_files(input_dir, output_dir):
    # Maak de output directory aan als die niet bestaat
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Doorloop alle tekstbestanden in de input directory
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.txt'):
            file_base_name = os.path.splitext(file_name)[0]
            input_file_path = os.path.join(input_dir, file_name)

            # Maak een subdirectory aan met de naam van het tekstbestand
            subdir_path = os.path.join(output_dir, file_base_name)
            if not os.path.exists(subdir_path):
                os.makedirs(subdir_path)

            # Lees de URLs en download de afbeeldingen
            with open(input_file_path, 'r') as file:
                urls = file.readlines()

            for idx, url in enumerate(urls):
                try:
                    url = url.strip()
                    response = requests.get(url, timeout=10)

                    # Check of de HTTP-response een afbeelding bevat
                    content_type = response.headers.get('Content-Type')
                    if 'image' not in content_type:
                        print(f"URL does not point to an image: {url}")
                        continue

                    image_content = response.content
                    image_file = io.BytesIO(image_content)

                    try:
                        image = Image.open(image_file)
                    except IOError:
                        print(f"Could not open image from {url}")
                        continue

                    # Converteer RGBA naar RGB als de afbeelding een alfakanaal heeft
                    if image.mode == 'RGBA':
                        image = image.convert("RGB")

                    image_file_path = os.path.join(subdir_path, f"{idx}.jpeg")
                    with open(image_file_path, "wb") as img_file:
                        image.save(img_file, "JPEG")

                    print(f"Downloaded: {image_file_path}")
                except requests.RequestException as req_err:
                    print(f"Network error for {url} - {req_err}")
                except Exception as e:
                    print(f"General error occurred for {url} - {e}")

#### User Input
Bij het uitvoeren van de webscraper worden alle tekst bestanden automatisch in een folder "img_src_paths" geplaatst. Dubbel check dat de gewilde bestanden met juiste links in deze folder zitten. Verder geef een output folder naam waarin nieuwe folders worden gemaakt me afbeeldingen opbasis de txt-bestand namen.

In [ ]:
input_directory = "img_src_paths" # Path naar txt bestanden
output_directory = "images" # Path naar bestemmings-folder

In [ ]:
download_images_from_files(input_directory, output_directory)

### Image-Sorter
Het gebruik van de `ImageSorter` class is optioneel, dit process kan ook handmatig gedaan worden maar voor het project is deze mogelijkheid wel gebruikt om de afbeeldingen te bekijken en soorteren.

In [ ]:
class ImageSorter:
    def __init__(self, image_dir):
        self.image_dir = image_dir
        self.image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir)]
        self.index = 0

        self.root = tk.Tk()
        self.root.title("Image Sorter")

        self.image_label = tk.Label(self.root)
        self.image_label.pack()

        self.failed_log_path = os.path.join(image_dir, 'failed_images.txt')

        self.load_image()

        self.root.bind("<Left>", lambda e: self.move_to_folder('goed'))
        self.root.bind("<Right>", lambda e: self.move_to_folder('weggooien'))
        self.root.bind("<Down>", lambda e: self.move_to_folder('edits'))

        self.root.mainloop()

    def load_image(self):
        try:
            image_path = self.image_paths[self.index]
            image = Image.open(image_path)
            photo = ImageTk.PhotoImage(image)
            self.image_label.config(image=photo)
            self.image_label.photo = photo
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            with open(self.failed_log_path, 'a') as log_file:
                log_file.write(f"Failed to load {image_path}: {e}\n")
            self.next_image()

    def next_image(self):
        self.index += 1
        if self.index < len(self.image_paths):
            self.load_image()
        else:
            self.root.quit()

    def move_to_folder(self, folder_name):
        target_folder = os.path.join(self.image_dir, folder_name)
        if not os.path.exists(target_folder):
            os.makedirs(target_folder)
        try:
            image_path = self.image_paths[self.index]
            os.rename(image_path, os.path.join(target_folder, os.path.basename(image_path)))
        except Exception as e:
            print(f"Error moving image {image_path}: {e}")
            with open(self.failed_log_path, 'a') as log_file:
                log_file.write(f"Failed to move {image_path}: {e}\n")
        self.next_image()

### Tot Slot
Als alles goed is verlopen heb je nu extra afbeeldingen gevonden voor je computer vision project. Zo niet raad ik je aan onderstaande links te bekijken:


- [Medium - towardsdatascience: Image Scraping with Python](https://towardsdatascience.com/image-scraping-with-python-a96feda8af2d) 
- [Google-webscraper on Github](https://github.com/techwithtim/AI-Web-Scraper)
- [Youtube Tutorial Image Webscraper](https://www.youtube.com/watch?v=NBuED2PivbY)